In [8]:
import sys
import os
import numpy as np
import importlib

# hacky way, figure out proper way later
import pickle as pkl
import re

In [2]:
sys.path.append(os.path.abspath("../"))

In [26]:
import utils.run

importlib.reload(utils.run)

<module 'utils.run' from '/Users/dkundu/Documents/phd/ann-rnn-modified/utils/run.py'>

In [28]:
train_run_id = "noisydata_with_choiced"  # new training run id
run_dir = "/Users/dkundu/Documents/phd/ann-rnn-modified/runs/"
train_run_dir = os.path.join(run_dir, train_run_id)
analyze_run_dir = os.path.join(train_run_dir, "analyze")

checkpoint_paths = [
    os.path.join(train_run_dir, file_path)
    for file_path in os.listdir(train_run_dir)
    if file_path.endswith(".pt")
]

checkpoints = np.sort(checkpoint_paths)

In [29]:
# silly but numeric sort for checkpoint files

numbered_files = [f for f in checkpoint_paths if re.search(r"\d+.pt$", f)]
sorted_files = sorted(
    numbered_files, key=lambda x: int(re.search(r"(\d+).pt$", x).group(1))
)

In [ ]:
for idx in range(1, 2):
    setup_results = utils.run.setup_analyze(train_run_id=train_run_id, sort_index=idx)

    run_envs_output = utils.run.run_envs(
        model=setup_results["model"], envs=setup_results["envs"]
    )
    fname_idx = int(
        checkpoints[idx]
        .rsplit("/")[-1]
        .rsplit("checkpoint_grad_steps=")[-1]
        .rstrip(".pt")
    )
    try:
        with open(
            f"Users/dkundu/Documents/phd/ann-rnn-modified/data/rnnmodels/noisydatachoiced{train_run_id}/analyze/{fname_idx}.pkl",
            "wb",
        ) as f:
            pkl.dump(run_envs_output, f)
    except Exception as e:
        print(e)